This notebook explores a simple hypothesis test checking whether the accuracy of a trained model for binary classificadtion is meaningfully different from a majority class baseline.  We test this making a parametric assumption: we assume that the binary correct/incorrect results follow a binomial distribution (and approximate the binomial with a normal distribution).

### Cell 2: Imports
This cell imports all the necessary libraries for the notebook.
* **sys, collections, math**: Standard Python libraries for system functions, data structures (like `Counter`), and mathematical operations.
* **pandas, numpy, scipy**: Core libraries for data science in Python. `pandas` is for data manipulation, `numpy` is for numerical operations, and `scipy` is for scientific and statistical computations, including sparse matrices and the normal distribution functions.
* **sklearn**: Scikit-learn is the primary machine learning library used here for preprocessing and implementing the logistic regression model.

In [ ]:
# Import the sys module for system-specific parameters and functions.
import sys
# Import the Counter class for counting hashable objects, like labels.
from collections import Counter
# Import preprocessing tools from scikit-learn.
from sklearn import preprocessing
# Import linear models from scikit-learn, specifically for Logistic Regression.
from sklearn import linear_model
# Import pandas for data manipulation, though not directly used in these functions.
import pandas as pd
# Import sparse matrix tools from scipy, essential for efficient feature storage.
from scipy import sparse
# Import numpy for numerical operations, especially array manipulation.
import numpy as np
# Import the square root function from the math module.
from math import sqrt 
# Import the normal distribution object from scipy.stats for statistical tests.
from scipy.stats import norm

### Cell 3: Data Reading Function
This cell defines a function `read_data` to load text data from a tab-separated values (`.tsv`) file. It iterates through each line, splitting it into a label and a text body, and appends them to two separate lists, `X` (features, i.e., text) and `Y` (labels).

In [ ]:
# Define a function to read data from a file.
def read_data(filename):
    # Initialize an empty list to store the text data (features).
    X=[]
    # Initialize an empty list to store the labels.
    Y=[]
    # Open the specified file with UTF-8 encoding.
    with open(filename, encoding="utf-8") as file:
        # Iterate over each line in the file.
        for line in file:
            # Remove trailing whitespace and split the line by tabs.
            cols=line.rstrip().split("\t")
            # The first column is the label.
            label=cols[0]
            # The second column is the text.
            text=cols[1]
            # Add the text to the feature list.
            X.append(text)
            # Add the label to the label list.
            Y.append(label)
    # Return the lists of features and labels.
    return X, Y

### Cell 4: Setting the Data Directory
This cell sets the file path for the directory containing the datasets. **This is a variable that the user must change** to point to the correct location on their own system.

In [ ]:
# Change this to the directory with your data (from the CheckData_TODO.ipynb exercise).  
# The directory should contain train.tsv, dev.tsv and test.tsv
# Set the string variable 'directory' to the path of the data folder.
directory="../data/text_classification_sample_data"

### Cell 5: Loading the Datasets
This cell calls the `read_data` function to load the training (`train.tsv`) and development (`dev.tsv`) sets into memory. The text and labels for each set are stored in separate variables.

In [ ]:
# Load the training data by calling the read_data function.
trainX, trainY=read_data("%s/train.tsv" % directory)
# Load the development (validation) data.
devX, devY=read_data("%s/dev.tsv" % directory)

### Cell 6: Majority Class Baseline Function
The `majority_class` function calculates the baseline accuracy. It works by finding the most common label in the training set (`trainY`) and then "predicting" that same label for every single item in the development set (`devY`). The accuracy of this simple strategy serves as our baseline for comparison.

In [ ]:
# Define a function to calculate the majority class baseline accuracy.
def majority_class(trainY, devY):
    # Create a Counter object to store frequencies of each label.
    labelCounts=Counter()
    # Iterate through the labels in the training set.
    for label in trainY:
        # Increment the count for the current label.
        labelCounts[label]+=1
    # Find the most common label and its count, and get the label name.
    majority=labelCounts.most_common(1)[0][0]
    
    # Initialize a counter for correct predictions to zero.
    correct=0.
    # Iterate through the labels in the development set.
    for label in devY:
        # Check if the development label matches the majority label from the training set.
        if label == majority:
            # If it matches, increment the correct counter.
            correct+=1
            
    # Print the majority class label and its accuracy on the dev set, formatted to 3 decimal places.
    print("%s\t%.3f" % (majority, correct/len(devY)))
    # Return the calculated baseline accuracy.
    return correct/len(devY)

### Cell 7: Dictionary-Based Feature Function
This cell defines a simple feature engineering function, `political_dictionary_feature`. It checks if words from a predefined "dictionary" (in this case, small sets of politically-associated words) appear in the input text. If a word is found, it creates a binary feature (e.g., `word_in_dem_dictionary: 1`).

In [ ]:
# Here's a sample dictionary we can create by inspecting the output of the Mann-Whitney test (in 2.compare/)
# Define a set of words associated with Democrats.
dem_dictionary=set(["republican","cut", "opposition"])
# Define a set of words associated with Republicans.
repub_dictionary=set(["growth","economy"])

# Define a function to create features based on these dictionaries.
def political_dictionary_feature(tokens):
    # Initialize an empty dictionary to store features for the current text.
    feats={}
    # Iterate over each token (word) in the input text.
    for word in tokens:
        # If the word is in the Democrat dictionary...
        if word in dem_dictionary:
            # ...set the corresponding feature to 1.
            feats["word_in_dem_dictionary"]=1
        # If the word is in the Republican dictionary...
        if word in repub_dictionary:
            # ...set its corresponding feature to 1.
            feats["word_in_repub_dictionary"]=1
    # Return the dictionary of features.
    return feats

### Cell 8: Unigram Feature Function
This function, `unigram_feature`, implements a common "bag-of-words" approach. For every word in the input text, it creates a unique binary feature (e.g., `UNIGRAM_economy: 1`). This is a more comprehensive but less targeted feature engineering method compared to the dictionary approach.

In [ ]:
# Define a function to create unigram (single word) features.
def unigram_feature(tokens):
    # Initialize an empty dictionary to store features.
    feats={}
    # Iterate over each word in the tokenized text.
    for word in tokens:
        # Create a feature named "UNIGRAM_<word>" and set its value to 1.
        feats["UNIGRAM_%s" % word]=1
    # Return the dictionary of unigram features.
    return feats

### Cell 9: Feature Building Function
The `build_features` function orchestrates the feature engineering process. It takes a list of documents and a list of feature functions (like the ones defined above). It applies every feature function to each document and aggregates the results, returning a list where each item is a dictionary of all features for a single document.

In [ ]:
# Define a function to build features for a dataset.
def build_features(trainX, feature_functions):
    # Initialize an empty list to hold the feature dictionaries for all documents.
    data=[]
    # Iterate through each document in the input data.
    for doc in trainX:
        # Initialize an empty dictionary for the current document's features.
        feats={}

        # The sample text data is already tokenized; if yours is not, do so here.
        # Split the document string into a list of tokens (words).
        tokens=doc.split(" ")
        
        # Iterate through the list of feature-generating functions provided.
        for function in feature_functions:
            # Call the function with the tokens and update the feats dictionary with the results.
            feats.update(function(tokens))

        # Add the completed feature dictionary for the document to the data list.
        data.append(feats)
    # Return the list of feature dictionaries.
    return data

### Cell 10: Vocabulary Creation Function
This function, `create_vocab`, builds a vocabulary that maps every unique feature name found in the training data to a unique integer ID. This mapping is a crucial step to convert our text-based features into a numerical format that machine learning models can understand.

In [ ]:
# This helper function converts a dictionary of feature names to unique numerical ids.
def create_vocab(data):
    # Initialize an empty dictionary to store the feature-to-ID mapping.
    feature_vocab={}
    # Initialize the ID counter to 0.
    idx=0
    # Iterate through each document's feature dictionary in the data.
    for doc in data:
        # Iterate through each feature name in the document's dictionary.
        for feat in doc:
            # If the feature has not been seen before...
            if feat not in feature_vocab:
                # ...add it to the vocabulary with the current unique ID.
                feature_vocab[feat]=idx
                # Increment the ID for the next new feature.
                idx+=1
                
    # Return the completed feature vocabulary.
    return feature_vocab

### Cell 11: Feature Vectorization Function
The `features_to_ids` function transforms the list of feature dictionaries into a `scipy.sparse.lil_matrix`. A sparse matrix is used because for any given document, most possible features will be absent (value of 0). This data structure saves memory by only storing the non-zero feature values.

In [ ]:
# This helper function converts a dictionary of feature names to a sparse representation
# that we can fit in a scikit-learn model.  This is important because almost all feature 
# values will be 0 for most documents (note: why?), and we don't want to save them all in 
# memory.

def features_to_ids(data, feature_vocab):
    # Create a new sparse matrix (LIL format) with dimensions (num_docs, num_features).
    new_data=sparse.lil_matrix((len(data), len(feature_vocab)))
    # Iterate through the data with both index and document's features.
    for idx,doc in enumerate(data):
        # Iterate through each feature in the current document's dictionary.
        for f in doc:
            # If the feature exists in our vocabulary (created from training data)...
            if f in feature_vocab:
                # ...set the value in the sparse matrix at the correct [row, column] coordinate.
                new_data[idx,feature_vocab[f]]=doc[f]
    # Return the populated sparse matrix.
    return new_data

### Cell 12: Full ML Pipeline Function
The `pipeline` function automates the entire process of training and evaluating a model. It builds features, creates a vocabulary (from the training data only, to prevent data leakage), vectorizes the data into sparse matrices, trains a `LogisticRegression` model, and prints its accuracy on the development set.

In [ ]:
# This function evaluates a list of feature functions on the training/dev data arguments
def pipeline(trainX, devX, trainY, devY, feature_functions):
    # Build feature dictionaries for the training data.
    trainX_feat=build_features(trainX, feature_functions)
    # Build feature dictionaries for the development data.
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data
    # Create a vocabulary mapping feature names to IDs, using only the training data.
    feature_vocab=create_vocab(trainX_feat)

    # Convert the training features to a sparse matrix of IDs.
    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    # Convert the development features to a sparse matrix of IDs using the same vocabulary.
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    # Initialize a Logistic Regression model with specified parameters.
    logreg = linear_model.LogisticRegression(C=1.0, solver='lbfgs', penalty='l2', max_iter=10000)
    # Train the model on the training data.
    logreg.fit(trainX_ids, trainY)
    # Print the model's accuracy on the development data, formatted to 3 decimal places.
    print("Accuracy: %.3f" % logreg.score(devX_ids, devY))

### Cell 13: Evaluation Function
This `evaluate` function is very similar to `pipeline`, but instead of just printing the final accuracy score, it returns the model's raw predictions and the true labels from the development set. This output is necessary to perform the detailed statistical test in the next step.

In [ ]:
# This function trains a model and returns the predicted and true labels for test data
def evaluate(trainX, devX, trainY, devY, feature_functions):
    # Build feature dictionaries for the training data.
    trainX_feat=build_features(trainX, feature_functions)
    # Build feature dictionaries for the development data.
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data
    # Create the vocabulary using only training set features.
    feature_vocab=create_vocab(trainX_feat)

    # Convert training and development features to sparse matrices.
    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    # Initialize a Logistic Regression model.
    logreg = linear_model.LogisticRegression(C=1.0, solver='lbfgs', penalty='l2', max_iter=10000)
    # Train the model on the training data.
    logreg.fit(trainX_ids, trainY)
    # Generate predictions for the development set.
    predictions=logreg.predict(devX_ids)
    # Return the model's predictions and the actual ground truth labels.
    return (predictions, devY)

### Cell 14: Calculate Baseline
This cell executes the `majority_class` function defined earlier to compute the baseline accuracy. This value is stored in the `baseline` variable and will be used as the null hypothesis ($H_0$) in our statistical test.

In [ ]:
# Calculate the majority class baseline accuracy and store it in the 'baseline' variable.
baseline=majority_class(trainY,devY)

### Cell 15: Binomial Hypothesis Test Function
This is the core statistical function. `binomial_test` performs a two-tailed Z-test to see if the model's accuracy is significantly different from the baseline.

* **Null Hypothesis ($H_0$)**: The model's true accuracy is equal to the baseline accuracy.
* **Alternative Hypothesis ($H_a$)**: The model's true accuracy is not equal to the baseline accuracy.

It calculates the model's observed accuracy (`success_rate`), the standard error for a proportion, the Z-score, the p-value, and a confidence interval. Finally, it compares the Z-score to the critical value (`z_alpha`) to decide whether to reject the null hypothesis.

In [ ]:
# Define a function to perform a statistical test (Z-test for proportions).
def binomial_test(predictions, truth, baseline, significance_level=0.95):
    # Initialize a list to store results (1 for correct, 0 for incorrect).
    correct=[]
    # Iterate through the predictions and true labels simultaneously.
    for pred, gold in zip(predictions, truth):
        # Append 1 if prediction is correct, 0 otherwise.
        correct.append(int(pred==gold))
        
    # Calculate the model's accuracy (mean of the 'correct' list).
    success_rate=np.mean(correct)

    # This is a two-tailed test, so we split the alpha level (1 - significance) between the two tails.
    critical_value=(1-significance_level)/2
    # Find the Z-score that corresponds to the critical value using the percent-point function (inverse CDF).
    # We multiply by -1 because ppf finds the left-tail value.
    z_alpha=-1*norm.ppf(critical_value)
    # Print the calculated critical value and the corresponding Z-score (z_alpha).
    print("Critical value: %.3f\tz_alpha: %.3f" % (critical_value, z_alpha))
    
    # The standard error is the square root of the variance / sample size.
    # The variance for a binomial distribution is p*(1-p).
    # We use the observed success_rate as our estimate for p.
    standard_error=sqrt((success_rate*(1-success_rate))/len(correct))

    # Calculate the Z-score: (observed_value - hypothesized_value) / standard_error.
    Z=(success_rate-baseline)/standard_error
    # Calculate the lower bound of the confidence interval.
    lower=success_rate-z_alpha*standard_error
    # Calculate the upper bound of the confidence interval.
    upper=success_rate+z_alpha*standard_error
    # Calculate the p-value for a two-tailed test using the cumulative distribution function (CDF) and multiply by 2.
    pval=norm.cdf(-abs(Z)) * 2
    # Print the model's accuracy and the sample size.
    print ("Accuracy: %.3f, n = %s" % (success_rate, len(correct)))
    # Print the calculated confidence interval.
    print("%s%% Confidence interval: [%.3f,%.3f]" % (significance_level*100, lower, upper))

    # Print the calculated Z-score.
    print("Z score: %.3f" % Z)
    # Print the calculated p-value.
    print("p-value: %.5f" % pval)

    # Print the critical region in terms of accuracy scores.
    print ("Critical region corresponding to z_alpha=[%.3f,%.3f]: [%.3f, %.3f]" % (-z_alpha, z_alpha, baseline-z_alpha*standard_error, baseline+z_alpha*standard_error))
    # Print the final conclusion: whether we can reject the null hypothesis based on the Z-score.
    print ("Can we reject null that %.3f is different from %.3f at %s significance level? %s" % (success_rate, baseline, significance_level*100, "Yes" if Z < -z_alpha or Z > z_alpha else "No"))

### Cell 16: First Experiment (Dictionary Features)
This cell runs the first full experiment. It uses only the simple `political_dictionary_feature` set. It calls `evaluate` to train a model and get predictions, then passes those predictions to `binomial_test` to see if this very basic model is significantly better than just guessing the majority class.

In [ ]:
# Define the list of feature functions to use for this experiment.
features=[political_dictionary_feature]
# Train a model and get predictions and true labels using the specified features.
predictions, truth=evaluate(trainX, devX, trainY, devY, features)
# Perform the binomial test to compare the model's accuracy against the baseline.
binomial_test(predictions, truth, baseline, significance_level=.95)

### Cell 17: Second Experiment (Unigram Features)
This cell runs the second experiment, this time using the more powerful `unigram_feature` set. It follows the same process: evaluate the model and then run the statistical test. The results of this test will likely show a much larger Z-score and smaller p-value, indicating a highly significant improvement over the baseline.

In [ ]:
# Define the list of feature functions for the second experiment.
features=[unigram_feature]
# Train the model with unigram features and get the predictions.
predictions, truth=evaluate(trainX, devX, trainY, devY, features)
# Perform the statistical test on the results of the unigram model.
binomial_test(predictions, truth, baseline, significance_level=.95)

### Cell 18: Empty Cell
This is an empty cell, often left for future tests or code.

In [ ]:
# This cell is intentionally left blank.